In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import gzip
from collections import defaultdict
import math
import sklearn
from surprise import SVD, Reader, Dataset
from surprise.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import random

2023-12-02 23:51:37.613045: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-02 23:51:37.613068: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-02 23:51:37.613096: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-02 23:51:37.618527: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [3]:
game_meta = []
users = []

In [4]:
for d in readGz("steam_games.json.gz"):
    game_meta.append(d)
for d in readGz("australian_users_items.json.gz"):
    #remove all user without purchase
    if len(d['items']) > 0:
        users.append(d)

In [5]:
def price_to_text(price):
    if type(price) == str:
        return "Free to Play"
    price = float(price)
    if price<10:
        return "low-price"
    if price<35:
        return "medium-price"
    return "high-price"

In [6]:
all_genre_tags = defaultdict(int)
game_tags = defaultdict(set)
for d in game_meta:
    keys = d.keys()
    if ('id' not in keys):
        #print(d)
        if (d['url'] == 'http://store.steampowered.com/app/200260'):
            g = 200260
        else:
            continue
    else:
        g = d['id']
    if 'price' not in keys:
        game_tags[g].add('Free to Play')
    else:
        game_tags[g].add(price_to_text(d['price']))
    if ('tags' in keys):      
        for tag in d['tags']:
            game_tags[g].add(tag)
    if ('genres' in keys):
        for genre in d['genres']:
            game_tags[g].add(genre)
    # if ('specs' in keys):
    #     for spec in d['specs']:
    #         game_tags[g].add(spec)
    for tag in game_tags[g]:
        all_genre_tags[tag] += 1

In [58]:
for d in game_meta:
    keys = d.keys()
    if ('id' in keys and d['id'] == '434570'):
        print(d)

{'publisher': 'Grunge Games LTD', 'genres': ['Action', 'Adventure', 'Casual'], 'app_name': 'Blood and Bacon', 'sentiment': 'Overwhelmingly Positive', 'title': 'Blood and Bacon', 'url': 'http://store.steampowered.com/app/434570/Blood_and_Bacon/', 'release_date': '2016-02-01', 'tags': ['Gore', 'Horror', 'Action', 'Funny', 'Multiplayer', 'Shooter', 'FPS', 'Co-op', 'Online Co-Op', 'Casual', 'Memes', 'Blood', 'Adventure', 'First-Person', 'Indie', 'Singleplayer', 'Survival Horror', 'Illuminati', 'Psychological Horror', 'Dating Sim'], 'reviews_url': 'http://steamcommunity.com/app/434570/reviews/?browsefilter=mostrecent&p=1', 'specs': ['Single-player', 'Multi-player', 'Online Multi-Player', 'Co-op', 'Online Co-op', 'Steam Achievements', 'Full controller support', 'Steam Trading Cards', 'Stats', 'Steam Leaderboards'], 'price': 0.99, 'early_access': False, 'id': '434570', 'developer': 'Big Corporation'}


In [161]:
game_purchased_count = defaultdict(int)
for d in users:
    items = d['items']
    for item in items:
        if (item['playtime_forever'] > 60):
            game_purchased_count[item['item_id']] += 1

In [162]:
game_to_keep = set()
for g in game_purchased_count:
    if (game_purchased_count[g]>100):
        game_to_keep.add(g)

In [163]:
len(game_to_keep)

1937

In [164]:
sorted_tags = sorted(list(all_genre_tags.keys()), key = lambda x: all_genre_tags[x], reverse = True)

In [165]:
def process_data(all_users, tag_to_keep):
    A_i = defaultdict(list)
    tags_kept = set(sorted_tags[:tag_to_keep])
    tags_kept_l = sorted_tags[:tag_to_keep]
    rare_bonus = {k:(lambda x: x*1.0/tag_to_keep*0.6-0.3 if x>tag_to_keep/2 else 0)(i) for i,k in enumerate(tags_kept)}
    games_no_tags = []
    for g in game_to_keep:
        tags = game_tags[g]
        tag_list = list(game_tags[g])
        new_tags = tags_kept.intersection(tags)
        if (len(new_tags) == 0):
            #print("no tag!!")
            games_no_tags.append(g)
            continue
        else:
            for tag in tags_kept_l:
                if tag not in tags:
                    A_i[g].append(0)
                else:
                    #using gamma transformation to transform ranking of tag to 0-1
                    relevance = 1-(tag_list.index(tag)*1.0/len(game_tags[g]))**3
                    #apply rare bonus
                    relevance += rare_bonus[tag]
                    if (relevance>1):
                        relevance = 1.0
                    A_i[g].append(relevance)
    games_w_tags = list(A_i.keys())
    games_w_tags_set = set(A_i.keys())
    interactions = []
    users_kept = []
    for u in all_users:
        num_recorded = 0
        for item in u['items']:
            if item['item_id'] in games_w_tags_set:
                num_recorded += 1
        if num_recorded>=10:
            users_kept.append(u)
    a = 0
    above_zero = lambda x: x if x > 0 else 0
    for u in tqdm(users_kept):
        for item in u['items']:
            if item['item_id'] in games_w_tags_set:
                if (item['playtime_forever'] > 120):
                    interactions.append((a, games_w_tags.index(item['item_id']), above_zero(np.log2(item['playtime_forever']/60))))
        a += 1
    A_i_matrix = np.zeros((tag_to_keep, len(games_w_tags)))
    for a,g in enumerate(games_w_tags):
        for b,relevance in enumerate(A_i[g]):
            A_i_matrix[b,a] = relevance
    return A_i_matrix, interactions, users_kept, games_w_tags

In [175]:
A_i_matrix_50, interactions_50, users_kept_50, games_w_tags_50 = process_data(users, 50)

  0%|          | 0/54667 [00:00<?, ?it/s]

In [176]:
data_30 = process_data(users, 30)

  0%|          | 0/54667 [00:00<?, ?it/s]

In [182]:
data_50 = process_data(users, 50)

  0%|          | 0/54667 [00:00<?, ?it/s]

In [178]:
data_100 = process_data(users, 100)

  0%|          | 0/54667 [00:00<?, ?it/s]

In [64]:
average_hours = float(sum([d[2] for d in interactions_50])*1.0/len(interactions_50))

3.709715928702185

In [185]:
class tagMF(tf.keras.Model):
    def __init__(self, F, lamb, A_i, mu, U, I):
        super(tagMF, self).__init__()
        self.alpha = tf.Variable(mu)
        self.P = tf.Variable(tf.random.normal([U,F],stddev=0.001)) #User factors, R^(U, F)
        self.Theta_i = tf.Variable(tf.random.normal([len(A_i),F],stddev=0.001)) #tag to factors, R^(T, F)
        self.A_i = tf.constant(A_i) #Item-tag relevance, R^(I, T)
        self.betaU = tf.Variable(tf.random.normal([U],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([I],stddev=0.001))
        self.lamb = lamb
        self.F = F

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p_u = tf.reshape(self.P[u], [1, -1])
        p = tf.linalg.matmul(p_u, tf.transpose(self.Theta_i))
        p = tf.squeeze(p)
        final_result = tf.reduce_sum(tf.multiply(p, self.A_i[:,i]))
        print(final_result)
        return self.alpha + self.betaU[u] + self.betaI[i] + final_result
        
    def predict_given_user(self, u, i):
        p_u = tf.reshape(tf.constant(u), [1, -1])
        p = tf.linalg.matmul(p_u, self.Theta_i)
        p = tf.squeeze(p)
        Theta_Ai = tf.linalg.matmul(tf.transpose(self.Theta_i), tf.reshape(self.A_i[:,i], [-1, 1]))
        Theta_Ai = tf.squeeze(Theta_Ai)
        final_result = tf.reduce_sum(tf.multiply(p, Theta_Ai))
        return final_result
        
    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.P**2) + tf.reduce_sum(self.Theta_i**2) + tf.reduce_sum(self.betaU**2) + tf.reduce_sum(self.betaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        p_u = tf.transpose(tf.nn.embedding_lookup(self.P, u))
        A = tf.gather(self.A_i, indices=i, axis=1)
        i_factors = tf.linalg.matmul(tf.transpose(self.Theta_i), A)
        pred = tf.reduce_sum(tf.multiply(p_u, i_factors),0)
        return self.alpha + beta_u + beta_i + pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        squared_diff = tf.square(pred - r)
        mse_loss = tf.reduce_mean(squared_diff)
        
        return mse_loss

In [186]:
def trainingStep(model, interactions):
    Nsamples = 50000
    reg = 0
    with tf.GradientTape() as tape:
        sampleU,sampleI,sampleR = zip(*random.sample(interactions, Nsamples))
        loss = model(sampleU,sampleI,sampleR)
        reg = model.reg()
        loss += reg
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()-reg

In [187]:
def MSE(y, ypred):
            return sum((yt - yp) ** 2 for yt, yp in zip(y, ypred)) / len(y)

In [219]:
training_result = {}
for tag_num, data_t in tqdm(zip([30,50,100], [data_30, data_50, data_100])):
    for lamb in [0.001, 0.0001]:
        for iter in range(10):
            random.seed(i)
            train_interaction, test_interaction = train_test_split(data_t[1], train_size=0.98, random_state=i)
            model = tagMF(30,0.0001,data_t[0].astype("float32"), average_hours, len(data_t[2]), len(data_t[3]))
            optimizer = tf.keras.optimizers.Adam(0.05)
            for i in tqdm(range(101)):
                obj = trainingStep(model, train_interaction)
                if (i%20 == 0):
                    sampleU,sampleI,sampleR =zip(*test_interaction)
                    predR = model.predictSample(sampleU,sampleI)
                    mse_vali = MSE(sampleR,predR)
                    print(obj)
                    print(f"MSE:{mse_vali}")
                    training_result[(tag_num, lamb, iter, i)] = (obj, mse_vali)

0it [00:00, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

tf.Tensor(3.7328007, shape=(), dtype=float32)
MSE:3.6267588138580322
tf.Tensor(2.6074855, shape=(), dtype=float32)
MSE:2.7972404956817627
tf.Tensor(2.4230843, shape=(), dtype=float32)
MSE:2.7243428230285645
tf.Tensor(2.3814702, shape=(), dtype=float32)
MSE:2.7237277030944824
tf.Tensor(2.3369083, shape=(), dtype=float32)
MSE:2.6769425868988037
tf.Tensor(2.253818, shape=(), dtype=float32)
MSE:2.6627514362335205


  0%|          | 0/101 [00:00<?, ?it/s]

tf.Tensor(3.7646694, shape=(), dtype=float32)
MSE:3.592909097671509
tf.Tensor(2.6057355, shape=(), dtype=float32)
MSE:2.757385730743408
tf.Tensor(2.4046195, shape=(), dtype=float32)
MSE:2.664794445037842
tf.Tensor(2.380111, shape=(), dtype=float32)
MSE:2.671818971633911
tf.Tensor(2.3146524, shape=(), dtype=float32)
MSE:2.6569418907165527
tf.Tensor(2.2915025, shape=(), dtype=float32)
MSE:2.646397829055786


...

tf.Tensor(3.7646554, shape=(), dtype=float32)
MSE:3.5917890071868896
tf.Tensor(2.472536, shape=(), dtype=float32)
MSE:2.7656397819519043
tf.Tensor(2.262566, shape=(), dtype=float32)
MSE:2.6819515228271484
tf.Tensor(2.1958423, shape=(), dtype=float32)
MSE:2.663058042526245
tf.Tensor(2.1548624, shape=(), dtype=float32)
MSE:2.650486469268799
tf.Tensor(2.12258, shape=(), dtype=float32)
MSE:2.6232755184173584


  0%|          | 0/101 [00:00<?, ?it/s]

tf.Tensor(3.764849, shape=(), dtype=float32)
MSE:3.592958688735962
tf.Tensor(2.4801068, shape=(), dtype=float32)
MSE:2.7557404041290283
tf.Tensor(2.2768102, shape=(), dtype=float32)
MSE:2.6768639087677
tf.Tensor(2.2165475, shape=(), dtype=float32)
MSE:2.6489806175231934
tf.Tensor(2.1706135, shape=(), dtype=float32)
MSE:2.6473731994628906
tf.Tensor(2.1452355, shape=(), dtype=float32)
MSE:2.618006944656372


  0%|          | 0/101 [00:00<?, ?it/s]

tf.Tensor(3.7646697, shape=(), dtype=float32)
MSE:3.5925934314727783
tf.Tensor(2.4827018, shape=(), dtype=float32)
MSE:2.7607598304748535
tf.Tensor(2.2890425, shape=(), dtype=float32)
MSE:2.666893482208252
tf.Tensor(2.2160578, shape=(), dtype=float32)
MSE:2.6472980976104736
tf.Tensor(2.1815567, shape=(), dtype=float32)
MSE:2.6427950859069824
tf.Tensor(2.129775, shape=(), dtype=float32)
MSE:2.60099720954895


In [226]:
training_average = {}
for tag_num, data_t in zip([30,50,100], [data_30, data_50, data_100]):
    for lamb in [0.001, 0.0001]:
        for i in [0,20,40,60,80,100]:
            result = 0
            for iter in range(10):
                result += training_result[(tag_num, lamb, iter, i)][1].numpy()
            training_average[(tag_num, lamb, i)] = result/10
training_average

{(30, 0.001, 0): 3.5960641860961915,
 (30, 0.001, 20): 2.770968699455261,
 (30, 0.001, 40): 2.663127326965332,
 (30, 0.001, 60): 2.673240041732788,
 (30, 0.001, 80): 2.6496713876724245,
 (30, 0.001, 100): 2.632195568084717,
 (30, 0.0001, 0): 3.5925827026367188,
 (30, 0.0001, 20): 2.7711607456207275,
 (30, 0.0001, 40): 2.6644452571868897,
 (30, 0.0001, 60): 2.6659401655197144,
 (30, 0.0001, 80): 2.648909664154053,
 (30, 0.0001, 100): 2.6273096561431886,
 (50, 0.001, 0): 3.5926591873168947,
 (50, 0.001, 20): 2.770169496536255,
 (50, 0.001, 40): 2.68430643081665,
 (50, 0.001, 60): 2.660231924057007,
 (50, 0.001, 80): 2.648494243621826,
 (50, 0.001, 100): 2.6229490995407105,
 (50, 0.0001, 0): 3.5917643308639526,
 (50, 0.0001, 20): 2.763747715950012,
 (50, 0.0001, 40): 2.680933427810669,
 (50, 0.0001, 60): 2.661432671546936,
 (50, 0.0001, 80): 2.6484280109405516,
 (50, 0.0001, 100): 2.6206633329391478,
 (100, 0.001, 0): 3.5933456420898438,
 (100, 0.001, 20): 2.765173006057739,
 (100, 0.001,

In [69]:
model.P[2].numpy()

array([ 0.19036959, -0.08997277,  0.09698071,  0.04347944,  0.20011514,
       -0.02755693, -0.0704846 , -0.14988087,  0.08751268, -0.04148639,
        0.0598563 ,  0.12788755,  0.22028668,  0.07064371, -0.12278533,
        0.00432835,  0.06285044, -0.2658008 ,  0.02103275,  0.08011886,
       -0.15937188,  0.04815554, -0.07468703,  0.02830835,  0.0029659 ],
      dtype=float32)

In [243]:
Theta_i = model.Theta_i.numpy()

In [246]:
Theta_i[:8,:10]

array([[ 0.08655585, -0.31059024,  0.34088385,  0.22870511,  0.28342706,
         0.16156828, -0.32745996, -0.5270682 , -0.35885203, -0.80280244],
       [-0.52094084, -0.00558778,  0.24897207,  0.21646231, -0.45195   ,
         0.29141492,  0.11563437,  1.3226572 , -0.10379349, -0.18671106],
       [ 0.03662562, -0.22050911, -0.01271682,  0.30618268,  0.6910591 ,
         0.03599494, -0.2281576 , -0.10191011,  0.07073135,  0.03287695],
       [ 0.11388445,  0.10570098,  0.26332816,  0.40769   , -0.1146616 ,
         0.16208254,  0.11109021,  0.11924891, -0.15399408,  0.00753842],
       [ 0.10828802, -0.13009731,  0.14095289, -0.17620967,  0.0103164 ,
        -0.4891214 , -0.31590685, -0.04466379, -0.31548688, -0.01409433],
       [-0.20929673, -0.07328509,  0.14584465, -0.0157699 , -0.6895931 ,
        -0.12830137,  0.5286995 , -0.4495328 ,  0.3801405 ,  0.19117741],
       [ 0.41861397, -0.6682073 ,  0.20890467, -0.2615907 ,  0.8585488 ,
        -0.01509964,  0.09487591, -0.83612597

In [254]:
Selected_tags = np.zeros(30).astype("float32")
Selected_tags[5] = 0.85
Selected_tags[6] = 0.95
Selected_tags[21] = 1.0

In [255]:
sorted_tags.index('Sci-fi')

21

In [256]:
sorted_tags[10]

'Singleplayer'

In [257]:
sorted_tags[:50]

['low-price',
 'Indie',
 'Action',
 'Adventure',
 'Casual',
 'Simulation',
 'Strategy',
 'RPG',
 'medium-price',
 'Free to Play',
 'Singleplayer',
 'Multiplayer',
 'Great Soundtrack',
 'Puzzle',
 'Early Access',
 '2D',
 'Atmospheric',
 'VR',
 'Sports',
 'Platformer',
 'Story Rich',
 'Sci-fi',
 'Fantasy',
 'Horror',
 'Open World',
 'Difficult',
 'Anime',
 'Massively Multiplayer',
 'Pixel Graphics',
 'Co-op',
 'Shooter',
 'Racing',
 'Female Protagonist',
 'Funny',
 'First-Person',
 'FPS',
 'Sandbox',
 'Arcade',
 'Retro',
 'Turn-Based',
 'Comedy',
 'Point & Click',
 'Survival',
 'high-price',
 'Classic',
 'Family Friendly',
 'Space',
 'Movie',
 'Gore',
 'Design & Illustration']

In [258]:
result = []
for i in range(len(games_w_tags_50)):
    result.append(model.predict_given_user(Selected_tags, i).numpy())

In [261]:
result_sorted = [(x,i) for i,x in enumerate(result)]
recommendation = sorted(result_sorted, reverse = True)[:20]
for rec in recommendation:
    print(f"https://store.steampowered.com/app/{games_w_tags_50[rec[1]]}   {rec[0]}")

https://store.steampowered.com/app/294100   11.195342063903809
https://store.steampowered.com/app/342200   10.85116958618164
https://store.steampowered.com/app/223830   10.40960693359375
https://store.steampowered.com/app/208140   10.259357452392578
https://store.steampowered.com/app/8500   10.23606014251709
https://store.steampowered.com/app/65980   10.105133056640625
https://store.steampowered.com/app/212680   10.065489768981934
https://store.steampowered.com/app/322910   9.968053817749023
https://store.steampowered.com/app/301520   9.942577362060547
https://store.steampowered.com/app/290300   9.89314079284668
https://store.steampowered.com/app/244160   9.84282112121582
https://store.steampowered.com/app/41800   9.807921409606934
https://store.steampowered.com/app/92100   9.799325942993164
https://store.steampowered.com/app/7760   9.697967529296875
https://store.steampowered.com/app/310380   9.58668041229248
https://store.steampowered.com/app/2820   9.522690773010254
https://store.st

In [228]:
class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, lamb):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(users_kept_50)],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(games_w_tags_50)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(users_kept_50),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(games_w_tags_50),K],stddev=0.001))
        self.lamb = lamb
        self.k = K

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i] +\
            tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
               tf.reduce_sum(self.betaI**2) +\
               tf.reduce_sum(self.gammaU**2) +\
               tf.reduce_sum(self.gammaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        pred = self.alpha + beta_u + beta_i +\
               tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        squared_diff = tf.square(pred - r)
        mse_loss = tf.reduce_mean(squared_diff)
        
        return mse_loss

In [239]:
training_result_lfm = {}
train_interaction, test_interaction = train_test_split(interactions_50, train_size=0.98, random_state=i)
model = LatentFactorModel(average_hours, 30, 0.001)
optimizer = tf.keras.optimizers.Adam(0.05)
random.seed(116)
for i in tqdm(range(101)):
    obj = trainingStep(model, train_interaction)
    if (i%20 == 0):
        print(obj)
        sampleU,sampleI,sampleR =zip(*test_interaction)
        predR = model.predictSample(sampleU,sampleI)
        print(f"MSE:{MSE(sampleR,predR)}")
        training_result_lfm[i] = (obj, MSE(sampleR,predR))

  0%|          | 0/101 [00:00<?, ?it/s]

tf.Tensor(3.6436248, shape=(), dtype=float32)
MSE:3.591348648071289
tf.Tensor(2.9753976, shape=(), dtype=float32)
MSE:2.930438280105591
tf.Tensor(2.8019717, shape=(), dtype=float32)
MSE:2.7960824966430664
tf.Tensor(2.7309363, shape=(), dtype=float32)
MSE:2.7315423488616943
tf.Tensor(2.7191393, shape=(), dtype=float32)
MSE:2.7124366760253906
tf.Tensor(2.7188134, shape=(), dtype=float32)
MSE:2.707327127456665
